In [1]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,89E7AA6C29227EFF,classic_bike,2021-02-12 16:14:56,2021-02-12 16:21:43,Glenwood Ave & Touhy Ave,525,Sheridan Rd & Columbia Ave,660,42.012701,-87.666058,42.004583,-87.661406,member
1,0FEFDE2603568365,classic_bike,2021-02-14 17:52:38,2021-02-14 18:12:09,Glenwood Ave & Touhy Ave,525,Bosworth Ave & Howard St,16806,42.012701,-87.666058,42.019537,-87.669563,casual


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49622 entries, 0 to 49621
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             49622 non-null  object 
 1   rideable_type       49622 non-null  object 
 2   started_at          49622 non-null  object 
 3   ended_at            49622 non-null  object 
 4   start_station_name  45576 non-null  object 
 5   start_station_id    45576 non-null  object 
 6   end_station_name    44264 non-null  object 
 7   end_station_id      44264 non-null  object 
 8   start_lat           49622 non-null  float64
 9   start_lng           49622 non-null  float64
 10  end_lat             49408 non-null  float64
 11  end_lng             49408 non-null  float64
 12  member_casual       49622 non-null  object 
dtypes: float64(4), object(9)
memory usage: 4.9+ MB


In [5]:
#We need to convert started_at and ended_at columns from object Dtype to datetime type so later we can extract year, month,day... and create datetime dimension table 
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [6]:
#now when we check we can see that we sucessfully changed Dtype of these two columns from object to datetime
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49622 entries, 0 to 49621
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ride_id             49622 non-null  object        
 1   rideable_type       49622 non-null  object        
 2   started_at          49622 non-null  datetime64[ns]
 3   ended_at            49622 non-null  datetime64[ns]
 4   start_station_name  45576 non-null  object        
 5   start_station_id    45576 non-null  object        
 6   end_station_name    44264 non-null  object        
 7   end_station_id      44264 non-null  object        
 8   start_lat           49622 non-null  float64       
 9   start_lng           49622 non-null  float64       
 10  end_lat             49408 non-null  float64       
 11  end_lng             49408 non-null  float64       
 12  member_casual       49622 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(7)
me

In [7]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

In [8]:
df['distance_in_km'] = df.apply(lambda row: haversine_distance(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis=1)

In [9]:
df = df.drop_duplicates().reset_index(drop=True)
df['rent_id'] = df.index

In the meantime we create data_model schema (saved as jpeg file in repository) and now we need to convert flat file (current dataframe) into star schema

In [10]:
datetime_dim = df[['started_at', 'ended_at']].reset_index(drop=True)
datetime_dim['started_at_datetime'] = datetime_dim['started_at']
datetime_dim['started_at_hour'] = datetime_dim['started_at_datetime'].dt.hour
datetime_dim['started_at_day'] = datetime_dim['started_at_datetime'].dt.day
datetime_dim['started_at_month'] = datetime_dim['started_at_datetime'].dt.month
datetime_dim['started_at_year'] = datetime_dim['started_at_datetime'].dt.year
datetime_dim['started_at_weekday'] = datetime_dim['started_at_datetime'].dt.weekday

datetime_dim['ended_at_datetime'] = datetime_dim['ended_at']
datetime_dim['ended_at_hour'] = datetime_dim['ended_at_datetime'].dt.hour
datetime_dim['ended_at_day'] = datetime_dim['ended_at_datetime'].dt.day
datetime_dim['ended_at_month'] = datetime_dim['ended_at_datetime'].dt.month
datetime_dim['ended_at_year'] = datetime_dim['ended_at_datetime'].dt.year
datetime_dim['ended_at_weekday'] = datetime_dim['ended_at_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index

datetime_dim = datetime_dim[['datetime_id','started_at_datetime','started_at_hour','started_at_day',
                            'started_at_month','started_at_year','started_at_weekday','ended_at_datetime',
                            'ended_at_hour','ended_at_day','ended_at_month','ended_at_year','ended_at_weekday']]

datetime_dim.head()


,datetime_id,started_at_datetime,started_at_hour,started_at_day,started_at_month,started_at_year,started_at_weekday,ended_at_datetime,ended_at_hour,ended_at_day,ended_at_month,ended_at_year,ended_at_weekday
0,0,2021-02-12 16:14:56,16,12,2,2021,4,2021-02-12 16:21:43,16,12,2,2021,4
1,1,2021-02-14 17:52:38,17,14,2,2021,6,2021-02-14 18:12:09,18,14,2,2021,6
2,2,2021-02-09 19:10:18,19,9,2,2021,1,2021-02-09 19:19:10,19,9,2,2021,1
3,3,2021-02-02 17:49:41,17,2,2,2021,1,2021-02-02 17:54:06,17,2,2,2021,1
4,4,2021-02-23 15:07:23,15,23,2,2021,1,2021-02-23 15:22:37,15,23,2,2021,1


In [11]:
ride_dim = df[['ride_id', 'rideable_type']].reset_index(drop=True)
ride_dim['ride_index'] = ride_dim.index
ride_dim = ride_dim[['ride_index','ride_id', 'rideable_type']]

ride_dim.head()

,ride_index,ride_id,rideable_type
0,0,89E7AA6C29227EFF,classic_bike
1,1,0FEFDE2603568365,classic_bike
2,2,E6159D746B2DBB91,electric_bike
3,3,B32D3199F1C2E75B,classic_bike
4,4,83E463F23575F4BF,electric_bike


In [12]:
start_station_dim = df[['start_station_id','start_station_name','start_lat','start_lng']].reset_index(drop=True)
start_station_dim['start_station_index'] = start_station_dim.index
start_station_dim = start_station_dim[['start_station_index','start_station_id','start_station_name','start_lat','start_lng']]
start_station_dim.head()

,start_station_index,start_station_id,start_station_name,start_lat,start_lng
0,0,525,Glenwood Ave & Touhy Ave,42.012701,-87.666058
1,1,525,Glenwood Ave & Touhy Ave,42.012701,-87.666058
2,2,KA1503000012,Clark St & Lake St,41.885795,-87.631101
3,3,637,Wood St & Chicago Ave,41.895634,-87.672069
4,4,13216,State St & 33rd St,41.834733,-87.625827


In [13]:
end_station_dim = df[['end_station_id','end_station_name','end_lat','end_lng']].reset_index(drop=True)
end_station_dim['end_station_index'] = end_station_dim.index
end_station_dim = end_station_dim[['end_station_index','end_station_id','end_station_name','end_lat','end_lng']]
end_station_dim.head()

,end_station_index,end_station_id,end_station_name,end_lat,end_lng
0,0,660,Sheridan Rd & Columbia Ave,42.004583,-87.661406
1,1,16806,Bosworth Ave & Howard St,42.019537,-87.669563
2,2,TA1305000029,State St & Randolph St,41.884866,-87.627498
3,3,TA1305000034,Honore St & Division St,41.903119,-87.673935
4,4,TA1309000055,Emerald Ave & 31st St,41.838163,-87.645123


In [14]:
fact_table = df.merge(datetime_dim,left_on='rent_id',right_on='datetime_id')\
.merge(ride_dim,left_on='rent_id',right_on='ride_index')\
.merge(start_station_dim,left_on='rent_id',right_on='start_station_index')\
.merge(end_station_dim,left_on='rent_id',right_on='end_station_index')\
[['member_casual','datetime_id','ride_index','start_station_index','end_station_index', 'distance_in_km']]

In [15]:
fact_table.head()

,member_casual,datetime_id,ride_index,start_station_index,end_station_index,distance_in_km
0,member,0,0,0,0,0.981104
1,casual,1,1,1,1,0.813412
2,member,2,2,2,2,0.315636
3,member,3,3,3,3,0.846501
4,member,4,4,4,4,1.643460


In [16]:
fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49622 entries, 0 to 49621
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   member_casual        49622 non-null  object 
 1   datetime_id          49622 non-null  int64  
 2   ride_index           49622 non-null  int64  
 3   start_station_index  49622 non-null  int64  
 4   end_station_index    49622 non-null  int64  
 5   distance_in_km       49408 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 2.3+ MB
